In [12]:
from data_analyze import *
import os, random, cv2, time, keras

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, Adamax
from keras import regularizers

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization

In [13]:
path = './archive/'
df = get_df(path)
df.head()

train_df, test_df = train_test_split(df, test_size=0.33, shuffle=True, random_state=123, stratify=df['class'])
test_df, val_df = train_test_split(test_df, test_size=0.5, shuffle=True, random_state=123, stratify=test_df['class'])

df_labels = {
    'Astrocitoma T1': 0,
    'Astrocitoma T1C+': 1 ,
    'Astrocitoma T2': 2,
    'Carcinoma T1': 4,
    'Carcinoma T1C+': 5,
    'Carcinoma T2': 6,
    'Ependimoma T1': 7,
    'Ependimoma T1C+': 8,
    'Ependimoma T2': 9,
    'Ganglioglioma T1': 10,
    'Ganglioglioma T1C+': 11,
    'Ganglioglioma T2': 12,
    'Germinoma T1': 13,
    'Germinoma T1C+': 14,
    'Germinoma T2': 15,
    'Glioblastoma T1': 16,
    'Glioblastoma T1C+': 17,
    'Glioblastoma T2': 18,
    'Granuloma T1': 19,
    'Granuloma T1C+': 20,
    'Granuloma T2': 21,
    'Meduloblastoma T1': 22,
    'Meduloblastoma T1C+': 23,
    'Meduloblastoma T2': 24,
    'Meningioma T1': 25,
    'Meningioma T1C+': 26,
    'Meningioma T2': 27,
    'Neurocitoma T1': 28,
    'Neurocitoma T1C+': 29,
    'Neurocitoma T2': 30,
    'Oligodendroglioma T1': 31,
    'Oligodendroglioma T1C+': 32,
    'Oligodendroglioma T2': 33,
    'Papiloma T1': 34,
    'Papiloma T1C+': 35,
    'Papiloma T2': 36,
    'Schwannoma T1': 37,
    'Schwannoma T1C+': 38,
    'Schwannoma T2': 39,
    'Tuberculoma T1': 40,
    'Tuberculoma T1C+': 41,
    'Tuberculoma T2': 42,
    '_NORMAL T1': 43,
    '_NORMAL T2': 44,
}

channels = 3
img_size = (224, 224)
img_shape = (img_size[0], img_size[1], channels)

def get_xy(df):
    X, y = [], [] # X = images, y = labels
    for ind in df.index:
        img = cv2.imread(str(df['image_path'][ind]))
        resized_img = cv2.resize(img, img_size) # Resizing the images to be able to pass on MobileNetv2 model
        X.append(resized_img) 
        y.append(df_labels[df['class'][ind]])

    X = np.array(X)
    y = np.array(y)
    X = X/255
    return X, y

X_train, y_train = get_xy(train_df)
X_test, y_test = get_xy(test_df)
X_val, y_val = get_xy(val_df)

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)
y_val = pd.get_dummies(y_val)

In [14]:
model = tf.keras.Sequential([
    Conv2D(filters=16, kernel_size=3, activation='relu', padding='same', input_shape=img_shape),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(44, activation='softmax')
])

model.compile(optimizer="adam", loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])

In [15]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
model.save('my_model.keras')
# model.evaluate(X_test,y_test)


Epoch 1/10
94/94 [==============================] - 36s 378ms/step - loss: 3.7388 - acc: 0.1140 - val_loss: 2.9096 - val_acc: 0.2622
Epoch 2/10
94/94 [==============================] - 34s 358ms/step - loss: 2.4373 - acc: 0.3753 - val_loss: 2.0409 - val_acc: 0.4743
Epoch 3/10
94/94 [==============================] - 34s 365ms/step - loss: 1.5406 - acc: 0.5827 - val_loss: 1.5384 - val_acc: 0.6189
Epoch 4/10
94/94 [==============================] - 34s 365ms/step - loss: 0.8964 - acc: 0.7657 - val_loss: 1.4276 - val_acc: 0.6149
Epoch 5/10
94/94 [==============================] - 35s 368ms/step - loss: 0.4925 - acc: 0.8640 - val_loss: 1.2158 - val_acc: 0.6973
Epoch 6/10
94/94 [==============================] - 34s 364ms/step - loss: 0.2884 - acc: 0.9233 - val_loss: 1.1888 - val_acc: 0.7135
Epoch 7/10
94/94 [==============================] - 34s 364ms/step - loss: 0.1858 - acc: 0.9500 - val_loss: 1.2414 - val_acc: 0.7108
Epoch 8/10
94/94 [==============================] - 34s 365ms/step - 

In [16]:
y_pred = model.predict(X_test, batch_size=64, verbose=1)

12/12 [==============================] - 2s 141ms/step


In [17]:
acc = pd.DataFrame({'train': history.history['acc'], 'val': history.history['val_acc']})

fig = px.line(acc, x=acc.index, y=acc.columns[0::], title='Training and Evaluation Accuracy every Epoch', markers=True)
fig.show()

loss = pd.DataFrame({'train': history.history['loss'], 'val': history.history['val_loss']})

fig = px.line(loss, x=loss.index, y=loss.columns[0::], title='Training and Evaluation Loss every Epoch', markers=True)
fig.show()